ratings1 и ratings2 — таблицы с данными о выставленных пользователями оценках фильмов. Они имеют одинаковую структуру и типы данных — на самом деле это две части одной таблицы с оценками фильмов.

userId — уникальный идентификатор пользователя, который выставил оценку;
movieId — уникальный идентификатор фильма;
rating — рейтинг фильма.
dates — таблица с датами выставления всех оценок.

date — дата и время выставления оценки фильму.
movies — таблица с информацией о фильмах.

movieId — уникальный идентификатор фильма;
title — название фильма и год его выхода;
genres — жанры фильма.

In [ ]:
import pandas as pd

ratings1_tab = pd.read_csv('data/ratings1.csv')
ratings2_tab = pd.read_csv('data/ratings2.csv')
dates_tab = pd.read_csv('data/dates.csv')
movies_tab = pd.read_csv('data/movies.csv')

In [ ]:
"""Задание 5.2 Сколько уникальных фильмов представлено в таблице movies?"""
movies_tab["title"].nunique()

In [ ]:
"""Задание 5.3 Сколько уникальных пользователей в таблице ratings1?"""
ratings1_tab["userId"].nunique()

In [ ]:
"""Задание 5.4 В каком году было выставлено больше всего оценок? Для ответа на этот вопрос используйте таблицу dates."""
dates_tab.head()
dates_tab['date'] = pd.to_datetime(dates_tab['date'])
popular_year = dates_tab['date'].dt.year.mode()
print(popular_year)


In [ ]:
"""склеим  ratings1 и ratings2 по строкам"""
"""concat является функцией библиотеки, а не методом DataFrame. Поэтому её вызов осуществляется как pd.concat(...)."""
ratings = pd.concat([ratings1_tab, ratings2_tab])
display(ratings)

In [ ]:
"""таблицы индексировались, начиная от 0. Чтобы создать новые индексы, нужно выставить параметр ignore_index на True:"""
ratings = pd.concat([ratings1_tab, ratings2_tab], ignore_index=True)
display(ratings)

In [ ]:
"""узнаем количество строк в таблицах ratings и dates, ведь нам предстоит вертикально склеить их между собой:"""
print('Число строк в таблице ratings: ', ratings.shape[0])
print('Число строк в таблице dates: ', dates_tab.shape[0])
print(ratings.shape[0] == dates_tab.shape[0])

In [ ]:
"""Чтобы очистить таблицу от дублей, мы можем воспользоваться методом DataFrame drop_duplicates(),
который удаляет повторяющиеся строки в таблице. Не забываем обновить индексы после удаления дублей,
выставив параметр ignore_index в методе drop_duplicates() на значение True:"""
ratings = ratings.drop_duplicates(ignore_index=True)
print('Число строк в таблице ratings: ', ratings.shape[0])

In [ ]:
"""конкатенируем таблицы ratings и dates по столбцам:"""
ratings_dates = pd.concat([ratings, dates_tab], axis=1)
display(ratings_dates.tail(7))

In [27]:
"""объединив таблицы типом left. Так как в наших таблицах есть одноимённые столбцы, установим один из суффиксов, чтобы избежать ошибки:"""
joined_false = ratings_dates.join(movies_tab, rsuffix='_right', how='left')
display(joined_false)

,userId,movieId,rating,date,movieId_right,title,genres
0,1,1,4.0,2000-07-30 18:45:03,1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2.0,Jumanji (1995),Adventure|Children|Fantasy
2,1,6,4.0,2000-07-30 18:37:04,3.0,Grumpier Old Men (1995),Comedy|Romance
3,1,47,5.0,2000-07-30 19:03:35,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance
4,1,50,5.0,2000-07-30 18:48:51,5.0,Father of the Bride Part II (1995),Comedy
...,...,...,...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22,NaN,NaN,NaN
100832,610,168248,5.0,2017-05-03 22:21:31,NaN,NaN,NaN
100833,610,168250,5.0,2017-05-08 19:50:47,NaN,NaN,NaN
100834,610,168252,5.0,2017-05-03 21:19:12,NaN,NaN,NaN


In [29]:
"""Чтобы совместить таблицы по ключевому столбцу с помощью метода join(), необходимо использовать ключевой столбец в «правой» таблице
в качестве индекса. Это можно сделать с помощью метода set_index(). Также необходимо указать название ключа в параметре on."""
joined = ratings_dates.join(movies_tab.set_index('movieId'), on='movieId', how='left')
display(joined.head())
"""В результате такого объединения для каждого идентификатора фильма movieId в таблице ratings_dates найден совпадающий
с ним идентификатор movieId в таблице movies и присоединена информация о самом фильме (title и genres). Это как раз то, что нам нужно.
Обратите внимание, что в результате такого объединения остался лишь один столбец movieId.
Примечание. Join() также можно использовать с параметром how='outer'."""

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [30]:
""" Метод merge() в первую очередь предназначен для слияния таблиц по заданным ключам,
поэтому он не требует установки ключевых столбцов в качестве индекса присоединяемой таблицы.
Кроме того, данный метод позволяет объединять даже таблицы с разноимёнными ключами.
Таким образом, merge() проще в использовании и более многофункционален, чем схожие методы."""
merged = ratings_dates.merge(movies_tab, on='movieId', how='left')
display(merged.head())

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [31]:
"""Проверим, что число строк в таблице ratings_dates совпадает с числом строк в результирующей таблице merged:"""
print('Число строк в таблице ratings_dates: ', ratings_dates.shape[0])
print('Число строк в таблице merged: ', merged.shape[0])
print(ratings_dates.shape[0] == merged.shape[0])

Число строк в таблице ratings_dates:  100836
Число строк в таблице merged:  100836
True


In [32]:
"""Задание 7.3 Дано две исходных таблицы:"""
data_1 = pd.DataFrame({'Value': [100, 45, 80], 'Group': [1, 4, 5]}, index = ['I0', 'I1', 'I2'])
data_2 = pd.DataFrame({'Company': ['Google', 'Amazon', 'Facebook'], 'Add': ['S0', 'S1', 'S7']}, index = ['I0', 'I1', 'I3'])
data_1.join(data_2, how='inner')

,Value,Group,Company,Add
I0,100,1,Google,S0
I1,45,4,Amazon,S1


In [35]:
"""Задание 7.4 Даны две исходные таблицы:"""
a = pd.DataFrame({'A': ['a', 'b', 'c'], 'B': [103, 214, 124], 'C': [1, 4, 2]})
b = pd.DataFrame({'V': ['d', 'b', 'c'], 'U': [1393.7, 9382.2, 1904.5], 'C': [1, 3, 2]})
a.merge(b, how='right', on='C')

,A,B,C,V,U
0,a,103.0,1,d,1393.7
1,NaN,NaN,3,b,9382.2
2,c,124.0,2,c,1904.5
